In [30]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sms-spam-collection-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F483%2F982%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240907%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240907T145150Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8272c9fa88c2361d7b46d0662bbff074bc4eace8446ecf90378d224b2d5976321bff36eaf8b0383daf9cd5e82582e475d2217e7f8a87eeeafccf0ea1ab3e6c10688f9c86a9b35375245e56f89e1bbe4d5b411d1395a59b0d69475e4a55d0239281ff5d481710ac569cbbc2cc27106d8e0945419304f07eba2690fa2da14ee83eed773aee8efa24850b9e662a4f9c13bf70c0e0578b28ed28656b99838eb9accb46db77c43d08a22cedb90353c121226a657eb0f86d3ad624d3bcbbf32c02ff6384bb539590076f4c4d2d2b3fbc707a07d0cf335ad710f4bb401343269323308d1a8c8be2321f1e1a266d9f42302e91a01b378c17657567687a3d150a4ca363e2'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 215934 bytes downloaded
Downloaded and uncompressed: sms-spam-collection-dataset
Data source import complete.


In [31]:
import pandas as pd


In [32]:
#Loading data
data = pd.read_csv("../input/sms-spam-collection-dataset/spam.csv", encoding='ISO-8859-1')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [33]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [34]:
# Check the column names
print(data.columns)


Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')


In [35]:
# Drop the Unnamed columns
data = data.drop(columns=["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"])

# Verify the changes
data.head()


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [36]:
print(data.columns)  # To see the actual column names
print(len(data.columns))  # To check the total number of columns


Index(['v1', 'v2'], dtype='object')
2


In [37]:
print(data.columns)


Index(['v1', 'v2'], dtype='object')


In [38]:
df =data[['v1','v2']]

In [39]:
df.rename(columns={'v1':'Label','v2':'messages'},inplace=True)

# Data Cleaning and Preprocessing

In [40]:
df.columns

Index(['Label', 'messages'], dtype='object')

In [41]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [43]:
corpus = []
for i in range (0,len(df)):
  review = re.sub('[^a-zA-Z]',' ',df['messages'][i])
  review = review.lower()
  review =review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review =' '.join(review)
  corpus.append(review)

In [44]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breather

In [45]:
import nltk
from nltk.stem import SnowballStemmer

In [46]:
snowball_ = SnowballStemmer('english')

In [47]:
corpus = []
for i in range (0,len(df)):
  review = re.sub('[^a-zA-Z]',' ',df['messages'][i])
  review = review.lower()
  review =review.split()
  review=[snowball_.stem(word) for word in review if not word in stopwords.words('english')]
  review =' '.join(review)
  corpus.append(review)

In [48]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkts st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breath

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 2500)

In [50]:
X = cv.fit_transform(corpus).toarray()

In [51]:
X.shape

(5572, 2500)

# Using the Binary in the convectorizer

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 2500,binary= True)

In [53]:
X = cv.fit_transform(corpus).toarray()

In [54]:
cv.vocabulary_

{'go': 786,
 'point': 1607,
 'crazi': 436,
 'avail': 139,
 'bugi': 264,
 'great': 810,
 'world': 2442,
 'la': 1075,
 'cine': 357,
 'got': 801,
 'wat': 2375,
 'ok': 1470,
 'lar': 1085,
 'joke': 1023,
 'wif': 2409,
 'oni': 1479,
 'free': 725,
 'entri': 599,
 'wkli': 2430,
 'comp': 391,
 'win': 2413,
 'cup': 450,
 'final': 682,
 'tkts': 2222,
 'st': 2067,
 'may': 1254,
 'text': 2181,
 'receiv': 1744,
 'question': 1703,
 'std': 2078,
 'txt': 2279,
 'rate': 1724,
 'appli': 98,
 'dun': 561,
 'say': 1871,
 'earli': 565,
 'alreadi': 66,
 'nah': 1384,
 'think': 2195,
 'goe': 789,
 'usf': 2321,
 'live': 1147,
 'around': 113,
 'though': 2201,
 'freemsg': 727,
 'hey': 865,
 'darl': 469,
 'week': 2389,
 'word': 2439,
 'back': 152,
 'like': 1132,
 'fun': 746,
 'still': 2080,
 'tb': 2156,
 'xxx': 2468,
 'send': 1902,
 'rcv': 1727,
 'even': 613,
 'brother': 254,
 'speak': 2049,
 'treat': 2257,
 'per': 1555,
 'request': 1784,
 'mell': 1271,
 'oru': 1500,
 'minnaminungint': 1294,
 'nurungu': 1452,
 'vet

# Using the n gram(unigram) in the countVectorizer

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 2500,binary= True,ngram_range=(1,2))
X = cv.fit_transform(corpus).toarray()

# Lemmetaization

In [60]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [61]:
 from nltk.corpus import stopwords
 from nltk.stem import WordNetLemmatizer
 wordlemmerize = WordNetLemmatizer()

In [62]:
corpus = []
for i in range (0,len(df)):
  review = re.sub('[^a-zA-Z]',' ',df['messages'][i])
  review = review.lower()
  review =review.split()
  review=[wordlemmerize.lemmatize(word) for word in review if not word in stopwords.words('english')]
  review =' '.join(review)
  corpus.append(review)

# Create TF-Idf and NGrams

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [64]:
tfidf = TfidfVectorizer(max_features=100)
X= tfidf.fit_transform(corpus).toarray()

In [65]:
import numpy as np
np.set_printoptions(edgeitems = 30 , linewidth=100000,
                    formatter = dict(float=lambda x: "%.3g" % x))

In [66]:
X

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.434, 0, 0, 0.462, 0.544, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.55, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.456, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0.473, 0, 0, 0, 0, 0, 0, 0, 0.492, 0, 0, 0, 0, 0, 0, 0, 0.572, 0, 0, 0, 0, 0, 0],
       [0.464, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.486, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.574, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# N-Gram in the Tfidf

In [67]:
tfidf = TfidfVectorizer(max_features=100,ngram_range = (2,2))
X= tfidf.fit_transform(corpus).toarray()

In [68]:
tfidf.vocabulary_

{'free entry': 32,
 'claim call': 16,
 'call claim': 3,
 'free call': 31,
 'chance win': 15,
 'txt word': 89,
 'let know': 53,
 'please call': 67,
 'lt gt': 57,
 'want go': 97,
 'like lt': 54,
 'sm ac': 79,
 'sorry call': 80,
 'call later': 8,
 'ur awarded': 90,
 'hi hi': 45,
 'call customer': 4,
 'customer service': 23,
 'cash prize': 14,
 'trying contact': 86,
 'draw show': 28,
 'show prize': 78,
 'prize guaranteed': 73,
 'guaranteed call': 41,
 'valid hr': 95,
 'selected receive': 75,
 'private account': 71,
 'account statement': 0,
 'statement show': 81,
 'call identifier': 5,
 'identifier code': 49,
 'code expires': 20,
 'urgent mobile': 94,
 'call landline': 7,
 'wat time': 98,
 'ur mob': 93,
 'gud ni': 42,
 'new year': 62,
 'send stop': 77,
 'co uk': 19,
 'nice day': 63,
 'lt decimal': 56,
 'decimal gt': 25,
 'txt nokia': 87,
 'good morning': 36,
 'ur friend': 92,
 'good night': 37,
 'camcorder reply': 12,
 'reply call': 74,
 'po box': 69,
 'last night': 52,
 'camera phone': 13,